# Debugging

## Disable JIT for Debugging

#### ```PYTORCH_JIT```

Setting the environment variable ```PYTORCH_JIT=0``` will disable all script and tracing annotations. If there is hard-to-debug error in one of the TorchScript models, you can use this flag to force everything to run native Python. Since TorchScript is disabled with this flag, we can use tools like ```pdb``` to debug the model code like this.

In [1]:
import torch

print(torch.__version__)

1.9.0+cu111


In [2]:
import pdb
    
@torch.jit.script
def scripted_fn(x: torch.Tensor):
    for i in range(12):
        x += x
    return x

def fn(x):
    x = torch.neg(x)
    pdb.set_trace()
    return scripted_fn(x)

traced_fn = torch.jit.trace(fn, (torch.rand(4, 5)))
traced_fn(torch.rand(3, 4))

> <ipython-input-2-6d7926d75035>(12)fn()
     10     x = torch.neg(x)
     11     pdb.set_trace()
---> 12     return scripted_fn(x)
     13 
     14 traced_fn = torch.jit.trace(fn, (torch.rand(4, 5)))

ipdb> x
tensor([[-0.7684, -0.5101, -0.7906, -0.4205, -0.9304],
        [-0.9736, -0.8795, -0.9005, -0.3209, -0.9383],
        [-0.0357, -0.7428, -0.5549, -0.5360, -0.9390],
        [-0.7503, -0.7314, -0.7051, -0.3489, -0.1526]])
ipdb> exit


BdbQuit: 

Debugging this script with ```pdb``` works except for when we invoke the ```@torch.jit.script``` function. We can globally disable JIT, so that we can call the ```@torch.jit.script``` function as a normal Python function and not compile it.

To disable the TorchScript Compiler for a specific function then ```@torch.jit.ignore```

## Inspecting Code

TorchScript provides a preety printer